## Glassdoor Job Scraper


In [237]:
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
import time
import csv
import pandas as pd

In [238]:
def webdriver_initialization(url):
    # The webdriver initialization
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.set_window_size(1120, 1000)
    driver.get(url)

In [239]:
def close_popup_window(driver):
    # Close popup glassdoor window
    try:
        driver.find_element(By.XPATH,'.//div[@class="d-flex flex-column pl-sm css-3g3psg css-1of6cnp e1rrn5ka4"]').click()
        time.sleep(1)
        driver.find_element(By.XPATH, './/span[@alt="Close"]').click()
        time.sleep(2)
        driver.find_element(By.XPATH, './/span[@alt="Close"]').click()
    except NoSuchElementException:
        pass   

In [240]:
def close_popup_window_1(driver):
    # Close popup glassdoor window
    try:
        driver.find_element(By.XPATH, './/span[@alt="Close"]').click()
    except NoSuchElementException:
        pass 

In [241]:
def reload_page_check(driver):
    try:
        driver.find_element(By.XPATH, './/button[@type="button"]').click()
        time.sleep(30)
        job_card.click()
    except NoSuchElementException:
        pass 
    
#<button type="button">Retry your search</button>

In [242]:
def scrape_job_card(driver, job_card):
    # Extract from single card job related data
    try:
        company_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@data-test="employerName"]'))).text
        #company_name = driver.find_element(By.XPATH,'.//div[@data-test="employerName"]').text
        
    except TimeoutException:
        company_name = "NI"
    try:
        position = driver.find_element(By.XPATH,'.//div[@data-test="jobTitle"]').text
    except NoSuchElementException:
        position = "NI"       
    try:
        location = driver.find_element(By.XPATH, './/div[@data-test="location"]').text
    except NoSuchElementException:
        location = "NI"
    try:
        salary = job_card.find_element(By.XPATH,'.//span[@data-test="detailSalary"]').text
    except NoSuchElementException:
        salary = "NI"    
    try:
        rating = driver.find_element(By.XPATH, './/span[@data-test="detailRating"]').text
    except NoSuchElementException:
        rating = "NI"
    try:
        description = driver.find_element(By.XPATH, './/div[@class="jobDescriptionContent desc"]').text
    except NoSuchElementException:
        description = "NI"
    try:
        company_overview = driver.find_element(By.XPATH, './/div[@class="d-flex flex-wrap"]').text
    except NoSuchElementException:
        company_overview = "NI"
    try:
        ratings_overview = driver.find_element(By.XPATH, './/ul[@class="css-38kpu8 erz4gkm0"]').text
    except NoSuchElementException:
        ratings_overview = "NI"
    try:
        pros_and_cons = driver.find_element(By.XPATH, './/div[@class="css-1xgpjcn e1vn3ovn5"]').text
    except NoSuchElementException:
        pros_and_cons = "NI"

    return position, company_name, location, rating, salary, description, company_overview, ratings_overview, pros_and_cons

In [243]:
def save_data_to_csv_file(records):
    # Save scraped data to csv file
    with open('jobs_info.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['position', 'company_name', 'location', 'rating', 'salary', 'description', 'company_overview', 'ratings_overview', 'pros_and_cons'])
        writer.writerows(records)

In [244]:
def glassdoor_scraper(url, num_job_cards, waiting_time):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.set_window_size(1120, 1000)
    driver.get(url)
    close_popup_window(driver)
    jobs = []
    while len(jobs) < num_job_cards:
        job_cards_list = driver.find_elements(By.XPATH,'.//div[@class="d-flex flex-column pl-sm css-3g3psg css-1of6cnp e1rrn5ka4"]')
        # Going through each job-card
        for i in range(len(job_cards_list)):
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_job_cards)))
            if len(jobs) >= num_job_cards:
                break
            job_cards_list[i].click()
            reload_page_check(job_cards_list[i])
            time.sleep(waiting_time)
            close_popup_window_1(driver)
            jobs.append(scrape_job_card(driver, job_cards_list[i]))
            if i == (len(job_cards_list)-1):
                # Clicking on the "next page" button
                driver.find_element(By.XPATH, './/span[@alt="next-icon"]').click()
                time.sleep(waiting_time)
    save_data_to_csv_file(jobs)

In [245]:
#url = "https://www.glassdoor.com/Job/germany-data-scientist-jobs-SRCH_IL.0,7_IN96_KO8,22.htm"
url = "https://www.glassdoor.co.uk/Job/data-scientist-jobs-SRCH_KO0,14.htm"
# Run a search function (it will open a new chrome window and start the scraping) 
glassdoor_scraper(url, num_job_cards=1110, waiting_time=3)

Progress: 0/1110
Progress: 1/1110
Progress: 2/1110
Progress: 3/1110
Progress: 4/1110
Progress: 5/1110
Progress: 6/1110
Progress: 7/1110
Progress: 8/1110
Progress: 9/1110
Progress: 10/1110
Progress: 11/1110
Progress: 12/1110
Progress: 13/1110
Progress: 14/1110
Progress: 15/1110
Progress: 16/1110
Progress: 17/1110
Progress: 18/1110
Progress: 19/1110
Progress: 20/1110
Progress: 21/1110
Progress: 22/1110
Progress: 23/1110
Progress: 24/1110
Progress: 25/1110
Progress: 26/1110
Progress: 27/1110
Progress: 28/1110
Progress: 29/1110
Progress: 30/1110
Progress: 31/1110
Progress: 32/1110
Progress: 33/1110
Progress: 34/1110
Progress: 35/1110
Progress: 36/1110
Progress: 37/1110
Progress: 38/1110
Progress: 39/1110
Progress: 40/1110
Progress: 41/1110
Progress: 42/1110
Progress: 43/1110
Progress: 44/1110
Progress: 45/1110
Progress: 46/1110
Progress: 47/1110
Progress: 48/1110
Progress: 49/1110
Progress: 50/1110
Progress: 51/1110
Progress: 52/1110
Progress: 53/1110
Progress: 54/1110
Progress: 55/1110
Pr

Progress: 437/1110
Progress: 438/1110
Progress: 439/1110
Progress: 440/1110
Progress: 441/1110
Progress: 442/1110
Progress: 443/1110
Progress: 444/1110
Progress: 445/1110
Progress: 446/1110
Progress: 447/1110
Progress: 448/1110
Progress: 449/1110
Progress: 450/1110
Progress: 451/1110
Progress: 452/1110
Progress: 453/1110
Progress: 454/1110
Progress: 455/1110
Progress: 456/1110
Progress: 457/1110
Progress: 458/1110
Progress: 459/1110
Progress: 460/1110
Progress: 461/1110
Progress: 462/1110
Progress: 463/1110
Progress: 464/1110
Progress: 465/1110
Progress: 466/1110
Progress: 467/1110
Progress: 468/1110
Progress: 469/1110
Progress: 470/1110
Progress: 471/1110
Progress: 472/1110
Progress: 473/1110
Progress: 474/1110
Progress: 475/1110
Progress: 476/1110
Progress: 477/1110
Progress: 478/1110
Progress: 479/1110
Progress: 480/1110
Progress: 481/1110
Progress: 482/1110
Progress: 483/1110
Progress: 484/1110
Progress: 485/1110
Progress: 486/1110
Progress: 487/1110
Progress: 488/1110
Progress: 48

Progress: 869/1110
Progress: 870/1110
Progress: 871/1110
Progress: 872/1110
Progress: 873/1110
Progress: 874/1110
Progress: 875/1110
Progress: 876/1110
Progress: 877/1110
Progress: 878/1110
Progress: 879/1110
Progress: 880/1110
Progress: 881/1110
Progress: 882/1110
Progress: 883/1110
Progress: 884/1110
Progress: 885/1110
Progress: 886/1110
Progress: 887/1110
Progress: 888/1110
Progress: 889/1110
Progress: 890/1110
Progress: 891/1110
Progress: 892/1110
Progress: 893/1110
Progress: 894/1110
Progress: 895/1110
Progress: 896/1110
Progress: 897/1110
Progress: 898/1110
Progress: 899/1110
Progress: 900/1110
Progress: 901/1110
Progress: 902/1110
Progress: 903/1110
Progress: 904/1110
Progress: 905/1110
Progress: 906/1110
Progress: 907/1110
Progress: 908/1110
Progress: 909/1110
Progress: 910/1110
Progress: 911/1110
Progress: 912/1110
Progress: 913/1110
Progress: 914/1110
Progress: 915/1110
Progress: 916/1110
Progress: 917/1110
Progress: 918/1110
Progress: 919/1110
Progress: 920/1110
Progress: 92

In [246]:
# Open csv file with the results and displays the first five rows of the results 
df = pd.read_csv("./jobs_info.csv")
df.head()

,position,company_name,location,rating,salary,description,company_overview,ratings_overview,pros_and_cons
0,Data Scientist (Remote),Pivotal\n4.2,"Cambridge, East of England, England",4.2,£78K - £93K (Glassdoor Est.),About Pivotal:\nWe are a team of ecologists an...,Size\n10000+ Employees\nFounded\n1998\nType\nC...,Career Opportunities\n★★★★★\n4.0\nComp & Benef...,"Pros\n""Great benefits, very flexible with its ..."
1,Junior Data Scientist,Oracle\n3.9,"London, England",3.9,£86K - £103K (Glassdoor Est.),At Oracle Health we are addressing the most co...,Size\n10000+ Employees\nFounded\n1977\nType\nC...,Career Opportunities\n★★★★★\n3.8\nComp & Benef...,"Pros\n""Good Work life Balance is there"" (in 12..."
2,Data Scientist,LEGO\n4.5,"London, England",4.5,£52K - £66K (Glassdoor Est.),Job Description\nCan you apply data science ac...,Size\n10000+ Employees\nFounded\n1932\nType\nC...,Career Opportunities\n★★★★★\n3.8\nComp & Benef...,"Pros\n""Salary comes on time but that's about i..."
3,Junior Data Scientist - London,Millennium Management LLC\n3.9,"London, England",3.9,£65K - £90K (Glassdoor Est.),Junior Data Scientist - London\nJob Summary\nW...,Size\n1001 to 5000 Employees\nFounded\n1989\nT...,Career Opportunities\n★★★★★\n3.9\nComp & Benef...,"Pros\n""Great people and culture."" (in 5 review..."
4,NI,NI,NI,NI,NI,NI,NI,NI,NI
